In [338]:
import re
import pandas as pd

df = pd.read_csv("C:/Users/Alyonka/Documents/spbu_docs_clean.csv")

df.head()


,Unnamed: 0,text,doc
0,0,268 дней осталось до 300-летия СПбГУ ...,-1000357493501362260.html
1,1,268 дней осталось до 300-летия СПбГУ ...,-1001667899699619798.html
2,2,268 дней осталось до 300-летия СПбГУ ...,-1001786768037123935.html
3,3,268 дней осталось до 300-летия СПбГУ ...,-1002479562235011756.html
4,4,268 дней осталось до 300-летия СПбГУ ...,-1003218939280141783.html


In [498]:
from collections import defaultdict
import pymorphy3
from nltk.corpus import stopwords
import math
from math import floor
from math import log

morph = pymorphy3.MorphAnalyzer()
stop_words = set(stopwords.words('russian'))

def normalize_text(text):
    # удаление знаков препинания и других неалфавитных символов
    text = re.sub(r'[^\w\t]+', ' ', text)
    # приведение к нижнему регистру
    text = text.lower()
    # разбиение текста на слова
    words = text.split()
    # лемматизация слов
    words = [morph.parse(word)[0].normal_form for word in words]
    # удаление стоп-слов
    words = [word for word in words if word not in stop_words]

    words = list(set(words))
    return words

'''def calculate_relevance(doc_id, query, words):
    doc_text = words[doc_id[0]]
    count = 0
    for word in query:
        count += doc_text.count(word)
    return count

def calculate_relevance_gamma(doc_id, query, words):
    doc_text = words[doc_id[0]]
    count = 0
    for word in query:
        count += doc_text.count(word)
    return count'''

'def calculate_relevance(doc_id, query, words):\n    doc_text = words[doc_id[0]]\n    count = 0\n    for word in query:\n        count += doc_text.count(word)\n    return count\n\ndef calculate_relevance_gamma(doc_id, query, words):\n    doc_text = words[doc_id[0]]\n    count = 0\n    for word in query:\n        count += doc_text.count(word)\n    return count'

In [403]:

def delta_encode(buffer):
    last = 0
    delta = []
    for i in range(len(buffer)):
        current = buffer[i]
        delta.append(current - last)
        last = current
    return delta

def delta_decode(buffer):
    last = 0
    d = buffer
    for i in range(len(buffer)):
        delta = d[i]
        d[i] = delta + last
        last = d[i]
    return d

In [340]:

import math
from math import floor
from math import log


def Binary_Representation_Without_MSB(x):
	binary = "{0:b}".format(int(x))
	binary_without_MSB = binary[1:]
	return binary_without_MSB

def EliasGammaEncode(k):
	if (k == 0):
		return '0'
	N = 1 + floor(log(k,2))
	Unary = (N-1)*'0'+'1'
	return Unary + Binary_Representation_Without_MSB(k)

def EliasDeltaEncode(x):
	if x==0: Gamma = EliasGammaEncode(0)
	else: Gamma = EliasGammaEncode(1 + floor(log(x,2)))
	binary_without_MSB = Binary_Representation_Without_MSB(x)
	return Gamma+binary_without_MSB


In [341]:
# Python3 implementation
from math import log

log2 = lambda x: log(x, 2)

def Unary(x):
	return (x-1)*'0'+'1'

def Binary(x, l = 1):
	s = '{0:0%db}' % l
	return s.format(x)
	
def Elias_Gamma(x):
	if(x == 0):
		return '0'

	n = 1 + int(log2(x))
	b = x - 2**(int(log2(x)))

	l = int(log2(x))

	return Unary(n) + Binary(b, l)
	
print(Elias_Gamma(1))


10


In [436]:
def  Elias_Delta_Decoding(x):
    t = list(x)
    if len(t) == 1 and t[0] == '0':
        return 0
    if len(t) == 1 and t[0] == '1':
        return 1
    L=0    
    while True:
        if not t[L] == '0':
            break
        L= L + 1
      
    # Reading L more bits and dropping ALL
    t=t[2*L+1:] 
      
    # Prepending with 1 in MSB
    t.insert(0,'1') 
    t.reverse()
    n=0
      
    # Converting binary to integer
    for i in range(len(t)): 
        if t[i]=='1':
            n=n+math.pow(2,i)
    return int(n)
  

In [343]:
def Elias_Gamma_Decoding(x):
    x = list(x)
    if len(x) == 1:
        return 0
    K = 0
    while True:
        if not x[K] == '0':
            break
        K = K + 1
      
    # Reading K more bits from '1'
    x = x[K:2*K+1]
  
    n = 0
    x.reverse()
      
    # Converting binary to integer
    for i in range(len(x)):
        if x[i] == '1':
            n = n+math.pow(2, i)
    return int(n)


Elias_Gamma_Decoding('0')

0

In [485]:

def inverted_index(tokens_list):
    # Создаем пустой словарь для инвертированного индекса
    inverted_index_dict = defaultdict(list)
    
    # Проходимся по всем документам в списке токенов
    for doc_id, tokens in enumerate(tokens_list):
        # Проходимся по всем токенам в документе
        for token in tokens:
                inverted_index_dict[token].append(doc_id)
    
    delta_index = {}

    for term in inverted_index_dict:
    # Создаем список для дельта-кодированных позиций
        delta_index[term] = delta_encode(inverted_index_dict[term])
    # Возвращаем инвертированный индекс в виде словаря
    return dict(delta_index)



In [505]:
def inverted_index_with_gamma(index):
    
    gamma_index = {}

    for term in index:
         gamma_values = []
         for n in index[term]:
             gamma_values.append(Elias_Gamma(n))
         gamma_index[term] = gamma_values
    return dict(gamma_index)


In [506]:
def inverted_index_with_delta(index):

    delta_elias_index = {}
    
    for term in index:
         delta_elias_values = []
         for n in index[term]:
             delta_elias_values.append(EliasDeltaEncode(n))
         delta_elias_index[term] = delta_elias_values
    return dict(delta_elias_index)

In [517]:
df_mini =  df.head(10000)

In [473]:
delta_decode([Elias_Delta_Decoding(x) for x in inverted_index_with_delta(df_mini.text.apply(normalize_text))['ректор']])

[0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 3,
 4,
 5,
 5,
 5,
 5,
 6,
 7,
 7,
 8,
 9,
 9,
 10,
 11,
 11,
 11,
 11,
 12,
 12,
 12,
 12,
 13,
 14,
 15,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 21,
 22,
 23,
 24,
 24,
 24,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 30,
 30,
 30,
 31,
 32,
 33,
 34,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 36,
 37,
 38,
 39,
 40,
 40,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 41,
 42,
 43,
 44,
 45,
 46,
 46,
 46,
 46,
 47,
 47,
 47,
 47,
 47,
 48,
 48,
 49,
 49,
 49,
 49,
 49,
 49,
 50,
 51,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 53,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 59,
 59,
 59,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 65,
 65,
 65,
 65,
 65,
 65,
 65,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 72,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 85,
 85,
 85,
 86,
 87,
 87,
 88,
 88,
 88,
 88,
 88,
 89,
 89,
 89,
 89,
 89,
 89,
 89,
 89,
 89,

In [518]:
inverted_index_dict = inverted_index(df_mini.text.apply(normalize_text))

In [519]:
import json

with open("index.json", 'w') as f:
    json.dump(inverted_index_dict, f)
    f.close()

In [523]:
d = {}
with open("index.json", 'r') as f:
    d = json.load(f)
    f.close()

d.keys()

dict_keys(['программа', 'арктический', 'наследие', 'blackboard', 'государственный', '15', '3', 'спорт', 'трансфер', 'информация', 'адамович', 'министерство', 'дополнительный', 'сайт', 'рамка', 'пройти', 'deasign', 'приём', 'студент', 'политика', 'обучаться', 'материал', '8', 'гражданин', 'специалитет', 'общественный', '300', 'медицинский', 'жизнь', 'консультация', 'противодействие', 'выставка', '14', 'набор', 'ректорский', 'выпускник', '19', 'личный', 'подготовка', 'гуманитарий', 'предыдущий', 'ассистент', 'здоровый', 'оборудование', 'набережная', 'лидер', 'pure', 'весна', 'защита', 'форум', 'многофункциональный', '2', 'ректор', 'лаборатория', 'лекция', 'экспортный', 'карьера', '268', 'страница', 'белорусский', 'кораблестроитель', 'ран', '2023', 'цифровой', 'vii', 'тестирование', 'криптовече', 'презентация', 'санкт', 'previous', 'грамотность', 'отдых', 'филолог', 'виртуальный', 'мегагрант', 'академический', 'март', 'интернет', 'язык', 'участвовать', 'next', 'труд', 'календарь', '7', 'п

In [471]:
t = normalize_text(df_mini.iloc[1].text)

indeces = [index for (index,item) in enumerate(t) if item =='ректор']

In [487]:
inverted_index_dict

{'программа': [0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'арктический': [0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
 

In [472]:
indeces

[51,
 194,
 312,
 321,
 358,
 366,
 396,
 456,
 479,
 874,
 887,
 940,
 1310,
 1456,
 1485,
 1904]

In [467]:
inverted_index_dict['ректор']

[0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1]

In [520]:
inverted_index_gamma = inverted_index_with_gamma(inverted_index_dict)

In [521]:
inverted_index_delta = inverted_index_with_delta(inverted_index_dict)

In [515]:
from collections import Counter

def index_search(query, index):
    query_words = normalize_text(query)
    if len(query_words) == 0:
        return('Stop word')
    try: result_docs = index[query_words[0]]
    except KeyError: return('Not Found') 
    for word in query_words:
        result_docs = list((Counter(result_docs) & Counter(index[word])).elements())
    result_docs = delta_decode(result_docs)
    #result_docs.sort(key=lambda x: calculate_relevance(x, query_words, index))
    return result_docs

def gamma_search(query, index):
    query_words = normalize_text(query)
    index[query_words[0]]
    if len(query_words) == 0:
        return('Stopword')
    try: result_docs = index[query_words[0]]
    except KeyError: return('Not Found') 
    for word in query_words:
        result_docs = list((Counter(result_docs) & Counter(index[word])).elements())
    result_docs = delta_decode([Elias_Gamma_Decoding(doc) for doc in list(result_docs)])
    return result_docs


def delta_search_(query, index):
    query_words = normalize_text(query)
    if len(query_words) == 0:
        return('Not found')
    try: result_docs = index[query_words[0]]
    except KeyError: return('Not Found') 
    for word in query_words:
        result_docs = list((Counter(result_docs) & Counter(index[word])).elements())
    result_docs = delta_decode([Elias_Delta_Decoding(doc) for doc in list(result_docs)])
    return result_docs


In [527]:
query = input("Введите запрос: ")
result_docs = index_search(query, inverted_index_dict)
print(result_docs)

[35, 70, 105, 261, 694, 703, 752, 957, 1041, 1245, 1366, 1530, 1612, 1743, 1846, 1955, 1982, 1987, 1992, 1997, 2057, 2233, 2402, 2405, 2408, 2424, 2440, 2464, 3008, 3019, 3213, 3414, 3508, 3653, 3686, 3693, 3912, 3953, 3999, 4086, 4111, 4283, 4356, 4431, 4506, 4510, 4663, 4797, 4940, 5137, 5213, 5302, 5391, 5603, 5611, 6039, 6283, 6474, 6582, 7230, 7301, 7528, 7623, 7821]


In [516]:
query = input("Введите запрос: ")
result_docs = delta_search_(query, inverted_index_delta)
print(result_docs)

[35, 468, 477, 526, 731]


In [528]:
query = input("Введите запрос: ")
result_docs = gamma_search(query, inverted_index_gamma)
print(result_docs)



[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,